In [2]:
# System and other standard libraries
import os
import pickle
import random
import time
import re
import itertools
import json
from pathlib import Path

# Data processing, metrics and plot libraries
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

# Pytorch and related libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.multiprocessing as mp
from torchinfo import summary

# Custom packages 
from utils import preprocessing as ppsr
from utils.potsimloader import potsimloader as psl
from utils import split
from models import linearregression, mlp, cnn, tcn, lstm, transformer
from utils import scaler
from training import train
from testing import evaluate

# Enabling polars global string cache, for more informaiton read below:
# https://docs.pola.rs/api/python/stable/reference/api/polars.StringCache.html
pl.enable_string_cache()

In [ ]:
def set_seed(seed):
    random.seed(seed)  # Set seed for Python's random module
    np.random.seed(seed)  # Set seed for NumPy
    
    # Setting seed for PyTorch devices
    torch.manual_seed(seed)  # For CPU
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)  # For current GPU (if using CUDA)
set_seed(42)

In [4]:
# Setting device type to be trained as "cuda" if aviablable else "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Generates Nitrogen(N) treatment as Npre/Npl-Nemg-Ntu 
# using the fertilizer values from 'n_values'
# such that (Npre/Npl + Nemg + Ntu) <= 400 kg/ha 
def generate_treatments(n_values):
    combis = list(itertools.product(n_values, repeat=3))
    return ["-".join(map(str, t)) for t in combis if sum(t) <= 400]


# Used to combine the yearly data from Dataverse into a 
# single complete file with '.parquet' extension named 
# 'potsim.parquet', saved to 'data' folder of the main directory
def join_potsim_yearly(data_dir, save_dir=Path("data"), save=True):
    if not save:
        return
    data_dir = Path(data_dir).resolve()
    files = os.listdir(data_dir)
    pattern = re.compile(r"^potsim_\d{4}\.parquet$")
    files = sorted([file for file in files if pattern.match(file)])
    files = [data_dir / file for file in files]
    df = pl.scan_parquet(files)
    filepath = save_dir / "potsim.parquet"
    df.sink_parquet(
        filepath,
        statistics=True,
        compression="zstd",
        compression_level=1,
        row_group_size=1_000_000,
    )

In [ ]:
# Columns that we would require to run the experiments.
# Modify in case of running custom experiments.
usecols = ['Year', 'Date', 'Treatment', 'NFirstApp','PlantingDay', 'IrrgDep',
           'IrrgThresh', 'DayAfterPlant', 'NApp', 'NLeach', 'NPlantUp', 'NTotL1', 
           'NTotL2', 'Irrg', 'SWatL1', 'SWatL2', 'Rain', 'SolarRad', 'AirTempC']
   
# For our training we only need data from one day before the first 
# application of fertilizer i.e. `Npre` or `Npl`. So, we create a 
# `mask` to filter the data later.
mask = (
    ((pl.col("NFirstApp") == "Npl") & (pl.col("DayAfterPlant") >= -1)) |
    ((pl.col("NFirstApp") == "Npre") & (pl.col("DayAfterPlant") >= -37))
)

In [ ]:
# Defining filepaths for all the data files
potsim_yearly_dir = Path("data/potsim_yearly")
weather_file = Path("data") / "weather.parquet"
data_file = Path("data") / "potsim.parquet"

# Change save=False if yearly potsim data already combined
join_potsim_yearly(potsim_yearly_dir, save_dir=Path("data"), save=True)

#### Defining the variables for the experiments
Modify according to your the requirments of the experiments
- `n_values`: The amount of N fertilizer to apply
- `scenario_filter`: The crop management scneraios we require to fetch from the actual dataset
- `(train/val/test)_years`: The respective years of the train/val/test split for experiments 

In [ ]:
# Possible values for n_values can be [0, 28, 56, 84, 112, 140, 168, 196]
n_values = [0, 56, 112, 168, 196]
treatments = generate_treatments(n_values)

# Please refer to the paper for all the possible values of each scenarios variable
scenario_filter= {
    "Year": list(range(2001, 2025)),
    "Treatment": treatments,
    "PlantingDay": [1, 15, 29, 43, 57],
    "IrrgDep": [30, 40, 50, 60],
    "IrrgThresh": [50, 60, 70, 80, 90],
    "NFirstApp": ["Npl", "Npre"]
}

# Divided as an exmaple, and can be adjusted as needed
train_years = list(range(2001, 2017))
val_years = list(range(2017, 2021))
test_years = list(range(2021, 2025))

For complete API documentation on `potsimloader` i.e. `psl` module, please refer to: [https://github.com/GatorSense/PotSim/tree/main/utils/potsimloader](https://github.com/GatorSense/PotSim/tree/main/utils/potsimloader)

In [ ]:
# Reads the combined `potsim.parquet` data and joins it with weather 
# data (if provided), then return a polars LazyFrame. 
data = psl.read_data(
    dataset_path=data_file,
    weather_path=weather_file,
    usecols=usecols,
    lazy=True,
    as_pandas=False,
)

# Uses the 'potsimloader` module to apply the scenario_filter`, selecting 
# the required scenarios. Returning a polars DataFrame.
data = psl.apply_filter(data, filters=scenario_filter, lazy=False, as_pandas=False)

# Applies the `mask` defined previously to further filter data for
# experiments. Then, converts and returns a pandas DataFrame
df = data.filter(mask).to_pandas()

#### Data Splitting Strategy

This section implements a **two-stage data splitting approach** that ensures both temporal and scenario-based separation between training, validation, and test sets. The splitting strategy prevents data leakage by maintaining distinct crop management practices across splits while respecting temporal boundaries. The `random_sample_train_val_test` function from the `utils/split` module  first groups the data by unique combinations of five management variables, then randomly assigns these groups to training, validation, and test sets based on set ratios *(60:20:20)*, making sure each split has different management scenarios regardless of year. After assigning scenarios, each split is filtered to include only data from the relevant years. This approach prevents overlap in management practices between splits and maintains clear separation by time, ensuring a fair evaluation of model performance.



<!--
**Stage 1: Scenario-Based Splitting**
The function first identifies unique combinations of crop management practices using five key variables: `Treatment`, `PlantingDay`, `IrrgDep`, `IrrgThresh`, and `NFirstApp`. These combinations are randomly shuffled and split according to the specified ratios (0.6:0.2:0.2), ensuring that *different management scenarios* are allocated to different splits regardless of year. For instance, if 100 unique combinations exist, 60 would be assigned to training, 20 to validation, and 20 to testing.

**Stage 2: Temporal Filtering**
After scenario assignment, the function applies temporal constraints by filtering each split (using `train_years`, `val_years`, and `test_years`) to include only data from the corresponding year ranges.

This two-stage approach ensures that models cannot exploit similarities in management practices between splits while maintaining temporal separation for realistic evaluation.
-->

In [ ]:
# Split data using stratified temporal approach
# - split: tuple of ratios (train, val, test) that must sum to 1.0
# - train_years/val_years/test_years: lists defining temporal boundaries for each split
# - seed: integer for reproducible random sampling of crop management combinations
train_split, val_split, test_split = split.random_sample_train_val_test(
    df,
    split=(0.6, 0.2, 0.2),
    train_years=train_years,
    val_years=val_years,
    test_years=test_years,
    seed = 42
)

#### Model Configuration and Feature Selection
This section configures the target variable and associated features for model training using a centralized configuration approach.

**Configuration Management:**
- All experimental configurations are defined in `utils/config.json`
- Change `target_col` to experiment with different prediction targets
- For custom experiments, modify `feature_cols` directly or update the configuration file at `utils/config.json`
- Override `feature_cols` with your own feature list for exploratory analysis
- Extend `utils/config.json` with new target variables and their corresponding feature sets

In [ ]:
# Select target variable for training/prediction
target_col = "NTotL1"

# Load experiment configuration from config file
# All target variables and their associated features are predefined in config.json
with open(Path("utils/config.json"), 'r') as file:
    metadata = json.load(file).get(target_col)
feature_cols = metadata["features"]

# Create output directory for model artifacts and results
output_dir = Path() / "outputs" / f"{target_col}"
output_dir.mkdir(parents=True, exist_ok=True)

# Define scaler save path for reproducible preprocessing
scaler_save_path =  output_dir / f"{target_col}_scaler.pkl"

#### Data Processing and Feature Engineering
This section processes the split datasets into model-ready tensors with appropriate scaling and sequence formatting for different model architectures.

**Key Processing Steps:**
- Missing values in numerical columns are filled with zeros
- Min-Max Normalization applied to prevent feature dominance
- For temporal models, creates sliding windows of length `seq_len`
- Outputs PyTorch tensors ready for model training

**Important Notes:**
- Always use `mode="fit"` only on training data to prevent data leakage
- Use `mode="transform"` for validation and test sets to apply the same scaling
- Adjust `seq_len` based on your temporal model requirements and data characteristics

In [ ]:
# Sequence length configuration:
# - Set to None for non-sequential models (LinearRegression, MLP, etc.)
# - Set to integer for sequential models (CNN, TCN, LSTM, etc.)
# seq_len = None  # For non-sequential models
seq_len = 15

# Process training data: fit scaler and transform features/targets
# Mode "fit" calculates scaling parameters from training data
X_train, y_train = ppsr.process_data(
    train_split,
    feats=feature_cols,
    tgt=target_col,
    scaler_path=scaler_save_path,
    mode="fit",
    seq_len=seq_len,
)

# Process validation data: apply existing scaler without refitting
# Mode "transform" uses previously fitted scaling parameters
X_val, y_val = ppsr.process_data(
    val_split,
    feats=feature_cols,
    tgt=target_col,
    scaler_path=scaler_save_path,
    mode="transform",
    seq_len=seq_len,
)

# Process test data: apply existing scaler without refitting
X_test, y_test = ppsr.process_data(
    test_split,
    feats=feature_cols,
    tgt=target_col,
    scaler_path=scaler_save_path,
    mode="transform",
    seq_len=seq_len,
)

In [ ]:
# Prints the shapes of each split
print(f"Train shapes: \nX_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Val shapes: \nX_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"Test shapes: \nX_test: {X_test.shape}, y_test: {y_test.shape}")

In [ ]:
# model_name = metadata['models']["Linear Regression"].get("class_name")
# model_params = metadata['models']["Linear Regression"].get("params")
# model = linearregression.LinearRegression(**model_params)
# model_state = output_dir / f"{target_col}_{model_name}.pth"

# model_name = metadata['models']["Multi-Layered Perceptron"].get("class_name")
# model_params = metadata['models']["Multi-Layered Perceptron"].get("params")
# model = mlp.MLP(**model_params)
# model_state = output_dir / f"{target_col}_{model_name}.pth"

# model_name = metadata['models']["1-D Convolutional Neural Network"].get("class_name")
# model_params = metadata['models']["1-D Convolutional Neural Network"].get("params")
# model = cnn.CNN1D(**model_params)
# model_state = output_dir / f"{target_col}_{model_name}.pth"

# model_name = metadata['models']["Temporal Convolutional Network"].get("class_name")
# model_params = metadata['models']["Temporal Convolutional Network"].get("params")
# model = tcn.TCN(**model_params)
# model_state = output_dir / f"{target_col}_{model_name}.pth"

model_name = metadata['models']["Long Short-Term Memory"].get("class_name")
model_params = metadata['models']["Long Short-Term Memory"].get("params")
model = lstm.LSTM(**model_params)
model_state = output_dir / f"{target_col}_{model_name}.pth"

# model_name = metadata['models']["Transformer"].get("class_name")
# model_params = metadata['models']["Transformer"].get("params")
# model = transformer.EncoderOnlyTransformer(**model_params, max_seq_len=seq_len)
# model_state = output_dir / f"{target_col}_{model_name}.pth"

In [ ]:
# Putting model to device "cuda" if GPU available else "cpu"
model.to(device)

# Print model summary
summary(model)

In [ ]:
# Used for calculating and printing metrics during training
min_tgt, max_tgt = scaler.get_min_max(tgt=target_col, scaler_path=scaler_save_path)
min_tgt = torch.tensor(min_tgt, device=device)
max_tgt = torch.tensor(max_tgt, device=device)
print(min_tgt, max_tgt)

#### Training Configuration and Hyperparameters
This section defines the training hyperparameters and optimization strategy for model training with early stopping and learning rate scheduling.

In [ ]:
# Core training hyperparameters
lr = 0.0001              # Learning rate - adjust based on model convergence
b_sz = 256               # Batch size - increase for larger datasets/GPU memory
max_epochs = 100          # Maximum training epochs

# Loss function and optimization
criterion = nn.MSELoss() 
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

# Learning rate scheduling for adaptive training
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=5, min_lr=1e-8
)

# Early stopping configuration
epochs_no_improve = 0
min_loss_reduction=1e-4
early_stop_patience: int = 10

# Mixed precision training for faster computation
ampscaler = torch.amp.GradScaler(device=device.type)

# Model and logging save paths
output_dir.mkdir(parents=True, exist_ok=True)
model_save_path = output_dir / f"{target_col}_{model_name}.pth"
logs_save_path = output_dir / f"logs_{target_col}_{model_name}.csv"

In [ ]:
#  Training DataLoader with data shuffling and performance optimization
train_loader = DataLoader(
    TensorDataset(X_train, y_train),
    num_workers=10,
    prefetch_factor=4,
    pin_memory=True,
    batch_size=b_sz,
    shuffle=True,
)

# Validation DataLoader without shuffling for consistent evaluation
val_loader = DataLoader(
    TensorDataset(X_val, y_val),
    num_workers=4,
    prefetch_factor=4,
    pin_memory=True,
    batch_size=b_sz,
    shuffle=False,
)

#### Model Training Loop
This section implements the main training loop with early stopping, learning rate scheduling, and comprehensive logging for model optimization. The below cell is commented,please uncomment if you are training.

In [1]:
# print(f"Training {model_name}on device: {device}")

# # Initialize training state variables
# best_val_loss = float("inf")
# train_val_logs = []
# early_stop = False

# for epoch in range(max_epochs):
#     # Check for early stopping condition
#     if early_stop:
#         print(f"Early stopping at epoch {epoch}")
#         break

#     start_time = time.time()

#     # Training phase with mixed precision scaling
#     train_loss, train_r2 = train.train_epoch(
#         model,
#         train_loader,
#         optimizer,
#         criterion,
#         device,
#         min_tgt,
#         max_tgt,
#         ampscaler=ampscaler,
#     )

#     # Validation phase without gradient computation
#     val_loss, val_r2 = train.val_epoch(
#         model, val_loader, criterion, device, min_tgt, max_tgt
#     )

#     # Update learning rate based on validation performance
#     scheduler.step(val_loss)

#     # Model checkpointing and early stopping logic
#     if val_loss < best_val_loss - min_loss_reduction:
#         best_val_loss = val_loss
#         torch.save(model.state_dict(), model_save_path)
#         epochs_no_improve = 0
#     else:
#         epochs_no_improve += 1
#         if epochs_no_improve > early_stop_patience:
#             early_stop = True

#     # Log training metrics and timing
#     elapsed = time.time() - start_time
#     curr_lr = optimizer.param_groups[0]["lr"]
#     train_val_logs.append(
#         {
#             "epoch": epoch + 1,
#             "train_loss": train_loss,
#             "val_loss": val_loss,
#             "train_r2": train_r2,
#             "val_r2": val_r2,
#             "learning_rate": curr_lr,
#         }
#     )

#     # Display epoch progress
#     print(
#         f"Epoch: {epoch + 1}/{max_epochs} | "
#         f"Train Loss: {train_loss:.5f}, Train R²: {train_r2:.4f} | "
#         f"Val Loss: {val_loss:.5f}, Val R²: {val_r2:.4f} | "
#         f"LR: {curr_lr}, Time: {round(elapsed, 2)} secs"
#     )

# # Ensure model is saved even if no improvement occurred
# if not model_save_path.exists():
#     torch.save(model.state_dict(), model_save_path)

# # Save training logs for analysis and visualization
# train_val_df = pd.DataFrame(train_val_logs)
# train_val_df.to_csv(logs_save_path, index=False)
# print("Training Complete....!")


### Training/Validation Loss Visualization
This section creates a comprehensive loss curve plot to monitor training progress and identify potential overfitting or convergence issues.

In [ ]:
# Load training logs and create loss curve visualization
df_log = pd.read_csv(logs_save_path)

# Create loss curve plot with training and validation losses
plt.figure(figsize=(8, 5))
sns.lineplot(x="epoch", y="train_loss", data=df_log, label="Train Loss")
sns.lineplot(x="epoch", y="val_loss", data=df_log, linestyle="--", label="Val Loss")

# Enhance plot readability and save
plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.savefig(output_dir / f"{target_col}_losscurve.png", dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
# Test DataLoader without shuffling for consistent evaluation
test_loader = DataLoader(
    TensorDataset(X_test, y_test),
    num_workers=4,
    prefetch_factor=4,
    pin_memory=True,
    batch_size=b_sz,
    shuffle=False,
)

In [ ]:
# Evaluate model performance on test data with multiple regression metrics
mae, rmse, nrmse, r2 = evaluate.evaluate_model(
    model,
    data_loader=test_loader,
    tgt=target_col,
    device=device,
    scaler_path=scaler_save_path,
)

# Display comprehensive evaluation metrics
print(f"\n\nMean Absolute Error: {mae:.4f}")
print(f"Root Mean Square Error: {rmse:.4f}")
print(f"Normalized Root Mean Square Error: {nrmse:.4f}")
print(f"Coefficient of determination (R-Squared): {r2:.4f}\n")